<a href="https://colab.research.google.com/github/txc-000/repo-3/blob/main/percobaan6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files
uploaded = files.upload()

Saving bbc_news.csv to bbc_news.csv


In [1]:
import pandas as pd
import tensorflow as tf

# Path ke file langsung
file_path = 'bbc_news.csv'

# Baca CSV dan ambil kolom title sebagai headline
headlines_df = pd.read_csv(file_path)
all_headlines = list(headlines_df.title.values)

print(f"Jumlah headline: {len(all_headlines)}")


Jumlah headline: 42115


In [2]:
print(all_headlines[:20])


['Ukraine: Angry Zelensky vows to punish Russian atrocities', 'War in Ukraine: Taking cover in a town under attack', "Ukraine war 'catastrophic for global food'", "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth", 'Ukraine conflict: Oil price soars to highest level since 2008', 'Ukraine war: PM to hold talks with world leaders on further sanctions', 'Ukraine war: UK grants 50 Ukrainian refugee visas so far', 'TikTok limits services as Netflix pulls out of Russia', "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales", 'Protests across Russia see thousands detained', 'Ukraine conflict: Your guide to understanding day 11', 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian', 'Ukraine: With placards and tears, Poles are greeting refugees like family', 'Twitter is part of our war effort - Ukraine minister', "Ukraine invasion: Volunteers 'working on autopilot'", 'Ukraine crisis: The West fights back agai

In [3]:
all_headlines[:20]

['Ukraine: Angry Zelensky vows to punish Russian atrocities',
 'War in Ukraine: Taking cover in a town under attack',
 "Ukraine war 'catastrophic for global food'",
 "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth",
 'Ukraine conflict: Oil price soars to highest level since 2008',
 'Ukraine war: PM to hold talks with world leaders on further sanctions',
 'Ukraine war: UK grants 50 Ukrainian refugee visas so far',
 'TikTok limits services as Netflix pulls out of Russia',
 "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales",
 'Protests across Russia see thousands detained',
 'Ukraine conflict: Your guide to understanding day 11',
 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian',
 'Ukraine: With placards and tears, Poles are greeting refugees like family',
 'Twitter is part of our war effort - Ukraine minister',
 "Ukraine invasion: Volunteers 'working on autopilot'",
 'Ukraine crisis: The West f

In [4]:
# Remove all headlines with the value of "Unknown"
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

42115

In [5]:
all_headlines[:20]

['Ukraine: Angry Zelensky vows to punish Russian atrocities',
 'War in Ukraine: Taking cover in a town under attack',
 "Ukraine war 'catastrophic for global food'",
 "Manchester Arena bombing: Saffie Roussos's parents on hearing the truth",
 'Ukraine conflict: Oil price soars to highest level since 2008',
 'Ukraine war: PM to hold talks with world leaders on further sanctions',
 'Ukraine war: UK grants 50 Ukrainian refugee visas so far',
 'TikTok limits services as Netflix pulls out of Russia',
 "Covid: Fourth jab for Scotland's vulnerable, and testing wind down fears in Wales",
 'Protests across Russia see thousands detained',
 'Ukraine conflict: Your guide to understanding day 11',
 'Russian gymnast investigated for wearing pro-war symbol on podium next to Ukrainian',
 'Ukraine: With placards and tears, Poles are greeting refugees like family',
 'Twitter is part of our war effort - Ukraine minister',
 "Ukraine invasion: Volunteers 'working on autopilot'",
 'Ukraine crisis: The West f

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None
)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize the words in our headlines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_headlines)
total_words = len(tokenizer.word_index) + 1
print('Total words: ', total_words)

Total words:  32637


In [8]:
# Print a subset of the word_index dictionary created by Tokenizer
subset_dict = {key: value for key, value in tokenizer.word_index.items() \
               if key in ['a','man','a','plan','a','canal','panama']}
print(subset_dict)

{'a': 8, 'man': 31, 'plan': 192, 'canal': 3979, 'panama': 8220}


In [9]:
tokenizer.texts_to_sequences(['a','man','a','plan','a','canal','panama'])

[[8], [31], [8], [192], [8], [3979], [8220]]

In [10]:
# Convert data to sequence of tokens
input_sequences = []
for line in all_headlines:
    # Convert our headline into a sequence of tokens
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Create a series of sequences for each headline
    for i in range(1, len(token_list)):
        partial_sequence = token_list[:i+1]
        input_sequences.append(partial_sequence)

print(tokenizer.sequences_to_texts(input_sequences[:5]))
input_sequences[:5]

['ukraine angry', 'ukraine angry zelensky', 'ukraine angry zelensky vows', 'ukraine angry zelensky vows to', 'ukraine angry zelensky vows to punish']


[[19, 1714],
 [19, 1714, 499],
 [19, 1714, 499, 662],
 [19, 1714, 499, 662, 1],
 [19, 1714, 499, 662, 1, 6103]]

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Determine max sequence length
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences with zeros at the beginning to make them all max length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         19, 1714], dtype=int32)

In [12]:
# Predictors are every word except the last
predictors = input_sequences[:,:-1]
# Labels are the last word
labels = input_sequences[:,-1]
labels[:5]

array([1714,  499,  662,    1, 6103], dtype=int32)

In [13]:
from tensorflow.keras import utils

labels = utils.to_categorical(labels, num_classes=total_words)

In [14]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Input is max sequence length - 1, as we've removed the last word for the label
input_len = max_sequence_len - 1

model = Sequential()

# Add input embedding layer
model.add(Embedding(total_words, 10, input_length=input_len))

# Add LSTM layer with 100 units
model.add(LSTM(100))
model.add(Dropout(0.1))

# Add output layer
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
import numpy as np

dummy_predictors = np.random.rand(100, predictors.shape[1]).astype(np.float32)
dummy_labels = np.random.rand(100, labels.shape[1]).astype(np.float32)

model.fit(dummy_predictors, dummy_labels, epochs=3, batch_size=8)


Epoch 1/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 107ms/step - loss: 169683.4531
Epoch 2/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - loss: 169652.9531
Epoch 3/3
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 169590.3125


In [18]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print(f"Total RAM: {ram_gb:.2f} GB")


Total RAM: 13.61 GB


In [19]:
from tensorflow.keras import backend as K
import gc

K.clear_session()
gc.collect()


0

In [ ]:
batch_size = 2  # atau 2 kalau masih crash
dataset = tf.data.Dataset.from_tensor_slices((predictors, labels))
dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

model.fit(dataset, epochs=30)


In [ ]:
import numpy as np

predictors = np.array(predictors, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)


In [ ]:
model.fit(predictors, labels, epochs=30, batch_size=8, verbose=1)

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
import numpy as np

def predict_next_token(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_class = np.argmax(predicted_probs, axis=-1)
    return predicted_class[0]  # hasil berupa 1 angka token ID

In [ ]:
model.fit(predictors, labels, epochs=30, verbose=1)